# Dublin Air Pollution
#### [ Programming for Data Analysis Project GMIT 2020 ]
<br>

<img align="left" width="340" src="http://airtoxenviro.com/wp-content/uploads/2015/05/Power-Plant-Clip-Art.png">

<br><br><br><br><br><br><br><br><br><br><br><br><br><br>
_Completed by:_ Olga Rozhdestvina (G00387844@gmit.ie)

___

#### General Set Up

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Colour scheme and styles for plots.
sns.set_theme(palette="Set2", context="paper", font_scale=1.5)

# Better sized plots.
plt.rcParams['figure.figsize'] = (10, 6)

I have collected data on air pollution in Dublin from 23/10/2020 to 21/10/2020 from [IQAir](https://www.iqair.com/) - a website of a Swiss air quality technology company. [1]

> 1. [IQAir: Air quality in Dublin](https://www.iqair.com/us/ireland/leinster/dublin)

In [3]:
# simulate the date range
rng = pd.date_range('23/10/2020', periods=30, freq='D')
rng

DatetimeIndex(['2020-10-23', '2020-10-24', '2020-10-25', '2020-10-26',
               '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30',
               '2020-10-31', '2020-11-01', '2020-11-02', '2020-11-03',
               '2020-11-04', '2020-11-05', '2020-11-06', '2020-11-07',
               '2020-11-08', '2020-11-09', '2020-11-10', '2020-11-11',
               '2020-11-12', '2020-11-13', '2020-11-14', '2020-11-15',
               '2020-11-16', '2020-11-17', '2020-11-18', '2020-11-19',
               '2020-11-20', '2020-11-21'],
              dtype='datetime64[ns]', freq='D')

In [4]:
AQI = [13,13,10,14,17,17,15,13,19,24,11,18,28,30,38,73,61,34,26,29,31,31,32,30,30,30,32,27,28,31]
AQI_mean = np.mean(AQI)
AQI_std = np.std(AQI)
AQI_mean, AQI_std

(26.833333333333332, 13.394235907865575)

In [5]:
PM25 = [3.1,3.2,2.4,3.4,4.2,4.1,3.7,3.2,4.5,5.7,2.6,4.3,6.7,7.2,9.3,22.5,16.8,8.2,6.3,2.8,2.5,2.9,3.7,3.0,2.6,1.7,2.0,3.4,3.1,2.7]
PM25_mean = np.mean(PM25)
PM25_std = np.std(PM25)
PM25_mean, PM25_std

(5.06, 4.364141763661365)

In [6]:
PM10 = [5.0,4.7,5.1,7.3,6.1,4.2,4.0,5.0,5.1,4.2,3.4,6.3,11.3,11.4,12.9,21.7,17.4,9.6,9.3,5.0,4.3,3.7,3.7,3.1,3.6,2.8,3.2,5.9,5.0,4.9]
PM10_mean = np.mean(PM10)
PM10_std = np.std(PM10)
PM10_mean, PM10_std

(6.640000000000001, 4.344160831890704)

In [7]:
o3 = [4.6,2.4,27.3,9.2,32.9,46.4,46.3,70.4,76.3,77.0,77.2,74.1,74.0,73.6,78.0,66.2,69.6,75.0]
o3_mean = np.mean(o3)
o3_std = np.std(o3)
o3_mean, o3_std

(54.47222222222223, 26.67911804213499)

In [8]:
NO2 = [21.2,10.5,12.1,8.9,10.9,8.7,7.3,19.4,5.2,4.1,8.4,15.9,2,6.8,30.2,34.2,39.7,15.9,15.2,19.7,6.7,5.6,8.1,6.9,6.0,9.7,6.1,6.7,17.5,10.6,2.1]
NO2_mean = np.mean(NO2)
NO2_std = np.std(NO2)
NO2_mean, NO2_std

(12.33225806451613, 8.92023387140155)

In [9]:
SO2 = [2.3,2.2,1.2,2.2,2.3,2.3,2.4,2.4,2.6,1.8,2.0,2.4,2.1,3.0,4.1,2.5,2.0,2.5,1.4,2.2,2.4,2.6,2.8,2.5,1.5,1.4,2.2,1.8,1.1]
SO2_mean = np.mean(SO2)
SO2_std = np.std(SO2)
SO2_mean, SO2_std

(2.213793103448276, 0.5805610472703193)